<img src="http://oproject.org/tiki-download_file.php?fileId=6&display">
<img src="http://gfif.udea.edu.co/root/tmva/img/tmva_logo.gif">
## Required header files

In [ ]:
#include <cstdlib>
#include <iostream>
#include <map>
#include <string>

#include "TChain.h"
#include "TFile.h"
#include "TTree.h"
#include "TString.h"
#include "TObjString.h"
#include "TSystem.h"
#include "TROOT.h"


#include "TMVA/Factory.h"
#include "TMVA/Tools.h"
#include "TMVA/DataLoader.h"
#include "TRInterface.h"
#include "TMVA/MethodC50.h"
#include "TMVA/MethodRSNNS.h"
#include "TMVA/MethodRSVM.h"
#include "TMVA/MethodRXGB.h"

#include<TMVA/efficiencies.h>
#include "TMVA/tmvaglob.h"
#include "TH2F.h"
#include "TFile.h"
#include "TIterator.h"
#include "TKey.h"


## ROOT-R Initialization 

In [ ]:
ROOT::R::TRInterface::Instance();
TMVA::Tools::Instance();

## Declare Factory and DataLoader


In [ ]:


TString outfileName( "RMVAOutput.root" );
TFile* outputFile = TFile::Open( outfileName, "RECREATE" );

TMVA::Factory *factory = new TMVA::Factory( "TMVAClassification", outputFile,
                                               "!V:ROC:Correlations:!Silent:Color:!DrawProgressBar:AnalysisType=Classification" );

TMVA::DataLoader *loader=new TMVA::DataLoader("mymc-dataset");

TFile *input = TFile::Open( "MyMCdataset.root");


## Adding variables to DataLoader

In [ ]:
loader->AddVariable( "var0", 'F' );
loader->AddVariable( "var1", 'F' );
loader->AddVariable( "var2", 'F' );
loader->AddVariable( "var3 := var0-var1", 'F' );
loader->AddVariable( "var4 := var0*var2", 'F' );
loader->AddVariable( "var5 := var1+var2", 'F' );   

## Setting up dataset from Trees

In [ ]:
std::cout << "--- TMVAClassification       : Using input file: " << input->GetName() << std::endl;
   
// --- Register the training and test trees
TTree *tsignal     = (TTree*)input->Get("MyMCSig");
TTree *tbackground = (TTree*)input->Get("MyMCBkg");
   
Double_t signalWeight     = 1.0;
Double_t backgroundWeight = 1.0;
   
loader->AddSignalTree    ( tsignal,     signalWeight     );
loader->AddBackgroundTree( tbackground, backgroundWeight );   
    
TCut mycuts = ""; 
TCut mycutb = ""; 

loader->fSignalWeight=signalWeight;
loader->fBackgroundWeight=backgroundWeight;
loader->fTreeS=tsignal;
loader->fTreeB=tbackground;
loader->PrepareTrainingAndTestTree( mycuts, mycutb,
                                        "nTrain_Signal=3000:nTrain_Background=3000:nTest_Signal=1449:nTest_Background=1449:SplitMode=Random:NormMode=NumEvents:!V" );
                                        

## Booking methods
Available Booking methods with options for RMVA are

C50 Boosted Decision Trees http://oproject.org/tiki-index.php?page=RMVA#C50Booking

RMLP Neural Networks http://oproject.org/tiki-index.php?page=RMVA#RSNNSMLP 

RSVM Support Vector Machines http://oproject.org/tiki-index.php?page=RMVA#E1071RSVM

Extreme Gradient Boosted (RXGB) Decision Trees http://oproject.org/tiki-index.php?page=RMVA#RXGBBooking

In [ ]:
//C50 Boosted Decision Trees (BDTs)
factory->BookMethod(loader, TMVA::Types::kC50, "C50","!H:NTrials=5:Rules=kTRUE:ControlSubSet=kFALSE:ControlBands=10:ControlWinnow=kFALSE:ControlNoGlobalPruning=kTRUE:ControlCF=0.25:ControlMinCases=2:ControlFuzzyThreshold=kTRUE:ControlSample=0:ControlEarlyStopping=kTRUE:!V" );
   
//Neural Networks using RSNNS package
factory->BookMethod(loader, TMVA::Types::kRSNNS, "RMLP","!H:VarTransform=N:Size=c(5):Maxit=10:InitFunc=Randomize_Weights:LearnFunc=Std_Backpropagation:LearnFuncParams=c(0.2,0):!V" );
   
//Support Vector Machines (SVM) using e1071
//factory->BookMethod(loader, TMVA::Types::kRSVM, "RSVM","!H:Kernel=linear:Type=C-classification:VarTransform=Norm:Probability=kTRUE:Tolerance=0.001:!V" );

//eXtreme Gradient Boosted XGB Decision Trees
factory->BookMethod(loader, TMVA::Types::kRXGB, "RXGB","!V:NRounds=50:MaxDepth=2:Eta=1" );

//TMVA BDTs
///factory->BookMethod(loader,TMVA::Types::kBDT, "BDT","!V:NTrees=10:MinNodeSize=2.5%:MaxDepth=2:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20" );


## Training 

In [ ]:
factory->TrainAllMethods();

## Testing

In [ ]:
factory->TestAllMethods();

factory->EvaluateAllMethods();    

// Save the output
outputFile->Close();

//    std::cout << "==> Wrote root file: " << outputFile->GetName() << std::endl;
std::cout << "==> TMVAClassification is done!" << std::endl;

delete factory;
delete loader;

In [ ]:
TFile *file=TFile::Open("RMVAOutput.root");
gDirectory->cd("mymc-dataset");

TMVA::plot_efficiencies(file,  2,gDirectory);
c->Draw();